# HOMEWORK 5: TEXT CLASSIFICATION
In this homework, you will create models to classify texts from TRUE call-center. There are two classification tasks:
1. Action Classification: Identify which action the customer would like to take (e.g. enquire, report, cancle)
2. Object Classification: Identify which object the customer is referring to (e.g. payment, truemoney, internet, roaming) 

In this homework, you are asked to do the following tasks:
1. Data Cleaning
2. Preprocessing data for keras
3. Build and evaluate a model for "action" classification
4. Build and evaluate a model for "object" classification
5. Build and evaluate a multi-task model that does both "action" and "object" classifications in one-go 


Note: we have removed phone numbers from the dataset for privacy purposes. 

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# import shutil
# shutil.copy("/content/drive/MyDrive/FRA 501 IntroNLP&DL/Dataset/clean-phone-data.csv", "/content/clean-phone-data.csv")

## 1. Import Libs

In [3]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
from IPython.display import display

import matplotlib.pyplot as plt

from collections import defaultdict

from pythainlp.tokenize import word_tokenize

from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model, load_model
from keras.layers import Input, Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

## 2. Loading data
First, we load the data from disk into a Dataframe.

A Dataframe is essentially a table, or 2D-array/Matrix with a name for each column.

In [ ]:
phone_df = pandas.read_csv('dataset/clean-phone-data.csv')

Let's preview the data.

In [ ]:
# Show the top 5 rows
display(phone_df.head())
# Summarize the data
phone_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


## 3. Data cleaning

We call the DataFrame.describe() again.
Notice that there are 33 unique labels/classes for object and 10 unique labels for action that the model will try to predict.
But there are unwanted duplications e.g. Idd,idd,lotalty_card,Lotalty_card

Also note that, there are 13389 unqiue sentence utterances from 16175 utterances. You have to clean that too!

### #TODO 1: 
You will have to remove unwanted label duplications as well as duplications in text inputs. 
Also, you will have to trim out unwanted whitespaces from the text inputs. 
This shouldn't be too hard, as you have already seen it in the demo.



In [ ]:
display(phone_df.describe())
display(phone_df.Object.unique())
display(phone_df.Action.unique())

,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

array(['enquire', 'report', 'cancel', 'Enquire', 'buy', 'activate',
       'request', 'Report', 'garbage', 'change'], dtype=object)

In [ ]:
# TODO1: Data cleaning
clean_df = phone_df.copy()

### 3.1 Trim white space

Remove space at starting and ending in sentences

In [ ]:
clean_df["Sentence Utterance_strip"] = (clean_df["Sentence Utterance"].str.strip()).str.lower().copy()

### 3.2 Fixing Label Duplication and Data Duplication

- Change all lable which has more than one to alphabet lower case.
- Remove some instance which that already appear in dataset.

In [ ]:
clean_df['clean_Action'] = clean_df['Action'].str.lower().copy()
clean_df['clean_Object'] = clean_df['Object'].str.lower().copy()

clean_df = clean_df.drop_duplicates("Sentence Utterance_strip", keep="first")

#visualize result
display(clean_df.head())
display(clean_df.describe())

print('Action------------------------')
display(clean_df.Action.unique())
display(clean_df.clean_Action.unique())
print([e for e in list(clean_df.Action.unique()) if e not in list(clean_df.clean_Action.unique())])

print('Object------------------------')
display(clean_df.Object.unique())
display(clean_df.clean_Object.unique())
print([e for e in list(clean_df.Object.unique()) if e not in list(clean_df.clean_Object.unique())])

,Sentence Utterance,Action,Object,Sentence Utterance_strip,clean_Action,clean_Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment,<phone_number_removed> ผมไปจ่ายเงินที่ counter...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อง...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโท...,report,phone_issues


,Sentence Utterance,Action,Object,Sentence Utterance_strip,clean_Action,clean_Object
count,13362,13362,13362,13362,13362,13362
unique,13362,10,32,13362,8,26
top,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,service,<phone_number_removed> ผมไปจ่ายเงินที่ counter...,enquire,service
freq,1,8540,2102,1,8643,2105


Action------------------------


array(['enquire', 'report', 'cancel', 'Enquire', 'buy', 'activate',
       'request', 'Report', 'garbage', 'change'], dtype=object)

array(['enquire', 'report', 'cancel', 'buy', 'activate', 'request',
       'garbage', 'change'], dtype=object)

['Enquire', 'Report']
Object------------------------


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'ringtone', 'rate',
       'loyalty_card', 'Idd', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nontruemove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd', 'garbage',
       'ringtone', 'rate', 'loyalty_card', 'contact', 'officer'],
      dtype=object)

['nonTrueMove', 'TrueMoney', 'Payment', 'Idd', 'Balance', 'Service', 'Loyalty_card']


## 4. Data Preparation

### #TODO 2: Preprocessing data for Keras
You will be using Tensorflow 2 keras in this assignment. Please show us how you prepare your data for keras.
Don't forget to split data into train and test sets (+ validation set if you want)

In [ ]:
# TODO2: Preprocessing data for Keras

### 4.1 Create dictionary for each label add in dataframe

In [ ]:
clean_finish_df = (clean_df.drop(labels=['Sentence Utterance', 'Action', 'Object'], axis=1)).copy()
display(clean_finish_df.head())
display(clean_finish_df.describe())

,Sentence Utterance_strip,clean_Action,clean_Object
0,<phone_number_removed> ผมไปจ่ายเงินที่ counter...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อง...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโท...,report,phone_issues


,Sentence Utterance_strip,clean_Action,clean_Object
count,13362,13362,13362
unique,13362,8,26
top,<phone_number_removed> ผมไปจ่ายเงินที่ counter...,enquire,service
freq,1,8643,2105


### 4.2 Action Labels and Object Labels
Create action map and object map

In [ ]:
#create numpy arrary for action and object labels
action_label = clean_finish_df.clean_Action.unique()
object_label = clean_finish_df.clean_Object.unique()

#Action map
action_to_num_map = dict(zip(action_label, range(len(action_label))))
num_to_action_map = dict(zip(range(len(action_label)), action_label))

#Object map
object_to_num_map = dict(zip(object_label, range(len(object_label))))
num_to_object_map = dict(zip(range(len(object_label)), object_label))

#Visualize to check
print("Action Map ->", len(action_to_num_map), "classes")
display(action_to_num_map)

print("Object Map", len(object_to_num_map), "classes")
display(object_to_num_map)

Action Map -> 8 classes


{'enquire': 0,
 'report': 1,
 'cancel': 2,
 'buy': 3,
 'activate': 4,
 'request': 5,
 'garbage': 6,
 'change': 7}

Object Map 26 classes


{'payment': 0,
 'package': 1,
 'suspend': 2,
 'internet': 3,
 'phone_issues': 4,
 'service': 5,
 'nontruemove': 6,
 'balance': 7,
 'detail': 8,
 'bill': 9,
 'credit': 10,
 'promotion': 11,
 'mobile_setting': 12,
 'iservice': 13,
 'roaming': 14,
 'truemoney': 15,
 'information': 16,
 'lost_stolen': 17,
 'balance_minutes': 18,
 'idd': 19,
 'garbage': 20,
 'ringtone': 21,
 'rate': 22,
 'loyalty_card': 23,
 'contact': 24,
 'officer': 25}

### 4.3 Prepare Dataset
- For training action classification
- For training object classification

In [ ]:
data = clean_finish_df.to_numpy()
print(data.shape)
display(data)

(13362, 3)


array([['<phone_number_removed> ผมไปจ่ายเงินที่ counter services เค้าเช็ต 3276.25 บาท เมื่อวานที่ผมเช็คที่ศูนย์บอกมียอด 3057.79 บาท',
        'enquire', 'payment'],
       ['internet ยังความเร็วอยุ่เท่าไหร ครับ', 'enquire', 'package'],
       ['ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ', 'report',
        'suspend'],
       ...,
       ['ยอดเงินเหลือเท่าไหร่ค่ะ', 'enquire', 'balance'],
       ['ยอดเงินในระบบ', 'enquire', 'balance'],
       ['สอบถามโปรโมชั่นปัจจุบันที่ใช้อยู่ค่ะ', 'enquire', 'package']],
      dtype=object)

In [ ]:
data[:,1] = np.vectorize(action_to_num_map.get)(data[:,1]) 
data[:,2] = np.vectorize(object_to_num_map.get)(data[:,2]) 
display(data)

array([['<phone_number_removed> ผมไปจ่ายเงินที่ counter services เค้าเช็ต 3276.25 บาท เมื่อวานที่ผมเช็คที่ศูนย์บอกมียอด 3057.79 บาท',
        0, 0],
       ['internet ยังความเร็วอยุ่เท่าไหร ครับ', 0, 1],
       ['ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ', 1, 2],
       ...,
       ['ยอดเงินเหลือเท่าไหร่ค่ะ', 0, 7],
       ['ยอดเงินในระบบ', 0, 7],
       ['สอบถามโปรโมชั่นปัจจุบันที่ใช้อยู่ค่ะ', 0, 1]], dtype=object)

#### - For training action classification
Prepare dataset for training action classification model has steps following:
- Find keywords : analyze keywords in text of each action class
- Create features : apply keyword to be features in dataset
- Split into training set and testing set

##### Find keywords

In [ ]:
def show_first_in_label(first, select_label, label_idx, num_map):
    print("Showing label \"%s\"" % num_map[select_label])
    select = data[data[:, label_idx] == select_label, 0]
    for i in range(min(first, len(select))):
        print(i, select[i])
    print("")

In [ ]:
first_three = 3
number_of_classes = 8
for i in range(number_of_classes):
    show_first_in_label(first_three, i, 1, num_to_action_map)

Showing label "enquire"
0 <phone_number_removed> ผมไปจ่ายเงินที่ counter services เค้าเช็ต 3276.25 บาท เมื่อวานที่ผมเช็คที่ศูนย์บอกมียอด 3057.79 บาท
1 internet ยังความเร็วอยุ่เท่าไหร ครับ
2 พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อง โกลไล

Showing label "report"
0 ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ
1 ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโทรออกไม่ได้คะ แต่เล่นเนตได้คะ
2 <phone_number_removed> ไปชำระค่าบริการช้า เน็ตตัด แต่พี่ไปชำระค่าบริการแล้วค่ะ แต่ยังใช้ไม่ได้ค่ะ

Showing label "cancel"
0 <phone_number_removed> จะยกเลิกข้อความนะคะ
1 <phone_number_removed> มี sms อะไรไม่รู้อ่ะค่ะ จะยกเลิกอ่ะค่ะ
2 <phone_number_removed> ใช้เบอร์เล่นอินเตอร์เน็ตอยู่ มันเป็นแอร์การ์ด จะยกเลิกอ่ะครับ

Showing label "buy"
0 <phone_number_removed> เป็นเติมเงินนะคะ เดี๋ยวนี้วันหมดอายุ ต้องขอต่ออีกหรือไม่คะ
1 internet 1k ที่สมัครไว้ใช้งานหมดแล้ว สามารถสมัครใหม่ได้หรือเปล่าครับ
2 sms ที่เคยสมัครไว้มันจะหายไปด้วยมั้ย <phone_number_removed> ที่สมัครเอาไว้เป็นหวยอ่ะครับ ที่สมัครไว้เห็นว่

In [ ]:
def create_keywords_dict(label_map, data, label_idx):
    keywords_dict = dict()

    for c in label_map.keys():
        sub_dict = defaultdict(lambda: 0)

        list_s = list(data[np.where(data[:,label_idx] == label_map[c])][:, 0])

        for s in list_s:
            sentence_tokenize = word_tokenize(s, engine="longest")

            for w in sentence_tokenize:
                sub_dict[w] = sub_dict[w]+1

        # sort sub dict
        sorted_sub_dict = {}
        sorted_keys = sorted(sub_dict, key=sub_dict.get)
        for w in sorted_keys:
            sorted_sub_dict[w] = sub_dict[w]

        keywords_dict[c] = sorted_sub_dict

    return keywords_dict

In [ ]:
action_keymap = create_keywords_dict(action_to_num_map, data, 1)

In [ ]:
def select_keywords(keywords_dict, top_key):
    list_keywords = []
    for c in keywords_dict.keys():
        if top_key < keywords_dict[c].keys():
            list_keywords += keywords_dict[c].keys()[:top_key]
        else:
            list_keywords += keywords_dict[c].keys()
    return list_keywords

In [ ]:
action_keywords = select_keywords(action_keymap, 3)

for analysis can see that ...

In [62]:
# action_keywords = ["ไม่ได้", "เท่าไหร", "ยังไง", "แต่", "หรอ", "ยกเลิก", "เลิก", "สัญญาณ", "สมัคร",
#                     "เปิด ", "ลงทะเบียน", "อินเตอร์เน็ต", "สอบถาม", "หรือไม่", "เปลี่ยน"]
action_list = [action_keywords for i in range(data.shape[0])]
action_key_arr = np.array(action_list)
print("number of action keywords:", len(action_keywords))

number of action keywords: 15


##### Create Feature

In [63]:
def gen_keywords_feature(string, keywords):
    result = np.zeros(len(keywords))
    for index, keyword in enumerate(keywords):
        if keyword in string:
            result[index] = 1
    return result

In [64]:
#input features in sentences
action_feature = np.array(list(map(gen_keywords_feature, data[:,0], action_key_arr)))
action_feature = action_feature.astype(int)
#output label in action classification
action_label = data[:, 1].astype(int)
action_label = to_categorical(action_label, num_classes=8)

print("Action Data")
print("Data shape", action_feature.shape)
print("label shape", action_label.shape)

Action Data
Data shape (13362, 15)
label shape (13362, 8)


In [65]:
idx_check = 20

In [66]:
data[idx_check,0]

'<phone_number_removed> มีการเปิดใช้งาน gprs หรือป่าว'

In [67]:
action_feature[idx_check]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [68]:
data[:idx_check,1]

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0, 1],
      dtype=object)

In [69]:
action_label[:idx_check]

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32)

##### Spilt into training set and testing set

In [70]:
action_x_train, action_x_test, action_y_train, action_y_test = train_test_split(action_feature, action_label,
                                                                                 train_size=0.8, random_state=27,
                                                                                 shuffle=True)

print(action_x_train.shape, action_y_train.shape)
print(action_x_test.shape, action_y_test.shape)

(10689, 15) (10689, 8)
(2673, 15) (2673, 8)


#### - For training object classification
Prepare dataset for training object classification model has steps following:
- Find keywords : analyze keywords in text of each action class
- Create features : apply keyword to be features in dataset
- Split into training set and testing set

##### Find keywords

In [71]:
first_three = 2
number_of_classes = 26
for i in range(number_of_classes):
    show_first_in_label(first_three, i, 2, num_to_object_map)

Showing label "payment"
0 <phone_number_removed> ผมไปจ่ายเงินที่ counter services เค้าเช็ต 3276.25 บาท เมื่อวานที่ผมเช็คที่ศูนย์บอกมียอด 3057.79 บาท
1 <phone_number_removed> เติมบัตรไม่ได้เลย

Showing label "package"
0 internet ยังความเร็วอยุ่เท่าไหร ครับ
1 <phone_number_removed> เป็นเติมเงินนะคะ เดี๋ยวนี้วันหมดอายุ ต้องขอต่ออีกหรือไม่คะ

Showing label "suspend"
0 ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ
1 <phone_number_removed> หมายเลขนี้โดนระงับบริการเหรอค่ะ

Showing label "internet"
0 พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อง โกลไล
1 <phone_number_removed> เค้าบอกจะส่งรหัสเน็ตมาให้ แต่ยังไม่ได้ส่งมาเลยค่ะ

Showing label "phone_issues"
0 ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโทรออกไม่ได้คะ แต่เล่นเนตได้คะ
1 <phone_number_removed> ไปชำระค่าบริการช้า เน็ตตัด แต่พี่ไปชำระค่าบริการแล้วค่ะ แต่ยังใช้ไม่ได้ค่ะ

Showing label "service"
0 *2222 ใช้งานยังไง ขอรายละเอียดการสมัครหน่อย
1 <phone_number_removed> จะยกเลิกข้อความนะคะ

Showing label "nontruemove"
0 <phone_number_re

In [72]:
object_keywords = ["ไม่ได้", "เท่าไหร", "ยังไง", "แต่", "หรอ", "ยกเลิก", "เลิก", "สัญญาณ", "สมัคร",
                    "เปิด ", "ลงทะเบียน", "อินเตอร์เน็ต", "สอบถาม", "หรือไม่", "เปลี่ยน"]
object_list = [object_keywords for i in range(data.shape[0])]
object_key_arr = np.array(object_list)
print("number of action keywords:", len(object_keywords))

number of action keywords: 15


##### Create Feature

In [73]:
#input features in sentences
object_feature = np.array(list(map(gen_keywords_feature, data[:,0], object_key_arr)))
object_feature = object_feature.astype(int)
#output label in action classification
object_label = data[:, 2].astype(int)
object_label = to_categorical(object_label, num_classes=26)

print("Action Data")
print("Data shape", object_feature.shape)
print("label shape", object_label.shape)

Action Data
Data shape (13362, 15)
label shape (13362, 26)


##### Split into training set and testing set

In [74]:
object_x_train, object_x_test, object_y_train, object_y_test = train_test_split(object_feature, object_label,
                                                                                 train_size=0.8, random_state=27,
                                                                                 shuffle=True)

print(object_x_train.shape, object_y_train.shape)
print(object_x_test.shape, object_y_test.shape)

(10689, 15) (10689, 26)
(2673, 15) (2673, 26)


## #TODO 3: Build and evaluate a model for "action" classification


In [75]:
#TODO 3: Build and evaluate a model for "action" classification

### Create Model

In [76]:
def action_Model():
    inputs = Input(shape=(action_x_train.shape[1], ), dtype='int32')
    
    x = Dense(units=32, activation='relu')(inputs)
    x = Dropout(rate=0.2)(x)
    x = Dense(units=16, activation='relu')(x)
    x = Dense(units=action_y_train.shape[1], activation='relu')(x)
    x = Activation('softmax')(x)

    return Model(inputs=inputs, outputs=x)

action_cls = action_Model()
action_cls.summary()
action_opt = Adam(learning_rate=0.01)
action_cls.compile(loss="categorical_crossentropy", optimizer=action_opt, metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 15)]              0         
                                                                 
 dense_26 (Dense)            (None, 32)                512       
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_27 (Dense)            (None, 16)                528       
                                                                 
 dense_28 (Dense)            (None, 8)                 136       
                                                                 
 activation_2 (Activation)   (None, 8)                 0         
                                                                 
Total params: 1176 (4.59 KB)
Trainable params: 1176 (4.59 K

### Train Model

In [77]:
action_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=1, min_lr=0.0001)
action_cls.fit(action_x_train, action_y_train, epochs=10, batch_size=8, callbacks=action_scheduler)

Epoch 1/10
1337/1337 [==============================] - 3s 2ms/step - loss: 1.2216 - accuracy: 0.6448 - lr: 0.0100
Epoch 2/10
1337/1337 [==============================] - 2s 2ms/step - loss: 1.2003 - accuracy: 0.6460 - lr: 0.0100
Epoch 3/10
1337/1337 [==============================] - 3s 2ms/step - loss: 1.1970 - accuracy: 0.6460 - lr: 0.0100
Epoch 4/10
1337/1337 [==============================] - 3s 2ms/step - loss: 1.1936 - accuracy: 0.6459 - lr: 0.0100
Epoch 5/10
1337/1337 [==============================] - 2s 2ms/step - loss: 1.1937 - accuracy: 0.6460 - lr: 0.0100
Epoch 6/10
1337/1337 [==============================] - 2s 2ms/step - loss: 1.1873 - accuracy: 0.6460 - lr: 1.0000e-03
Epoch 7/10
1337/1337 [==============================] - 2s 2ms/step - loss: 1.1869 - accuracy: 0.6460 - lr: 1.0000e-03
Epoch 8/10
1337/1337 [==============================] - 3s 2ms/step - loss: 1.1839 - accuracy: 0.6460 - lr: 1.0000e-03
Epoch 9/10
1337/1337 [==============================] - 2s 2ms/step 

### Evaluation

In [78]:
action_result = action_cls.evaluate(action_x_test, action_y_test)
print("test_loss:", action_result[0], "test_accuracy:", action_result[1])

84/84 [==============================] - 0s 2ms/step - loss: 1.1624 - accuracy: 0.6502
test_loss: 1.1624252796173096 test_accuracy: 0.6502057909965515


## #TODO 4: Build and evaluate a model for "object" classification



In [79]:
#TODO 4: Build and evaluate a model for "object" classification

### Create Model

In [80]:
def object_Model():
    inputs = Input(shape=(object_x_train.shape[1], ), dtype='int32')
    
    x = Dense(units=32, activation='relu')(inputs)
    x = Dropout(rate=0.2)(x)
    x = Dense(units=16, activation='relu')(x)
    
    x = Dense(units=object_y_train.shape[1], activation='relu')(x)
    x = Activation('softmax')(x)

    return Model(inputs=inputs, outputs=x)

object_cls = object_Model()
object_cls.summary()
object_opt = Adam(learning_rate=0.01)
object_cls.compile(loss="categorical_crossentropy", optimizer=object_opt, metrics=['accuracy'])

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 15)]              0         
                                                                 
 dense_29 (Dense)            (None, 32)                512       
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_30 (Dense)            (None, 16)                528       
                                                                 
 dense_31 (Dense)            (None, 26)                442       
                                                                 
 activation_3 (Activation)   (None, 26)                0         
                                                                 
Total params: 1482 (5.79 KB)
Trainable params: 1482 (5.79 K

### Train Model

In [81]:
object_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=1, min_lr=0.0001)
object_cls.fit(object_x_train, object_y_train, epochs=10, batch_size=8, callbacks=object_scheduler)

Epoch 1/10
1337/1337 [==============================] - 3s 2ms/step - loss: 2.8236 - accuracy: 0.2371 - lr: 0.0100
Epoch 2/10
1337/1337 [==============================] - 2s 2ms/step - loss: 2.8017 - accuracy: 0.2470 - lr: 0.0100
Epoch 3/10
1337/1337 [==============================] - 2s 2ms/step - loss: 2.7990 - accuracy: 0.2493 - lr: 0.0100
Epoch 4/10
1337/1337 [==============================] - 2s 2ms/step - loss: 2.7981 - accuracy: 0.2462 - lr: 0.0100
Epoch 5/10
1337/1337 [==============================] - 2s 1ms/step - loss: 2.7967 - accuracy: 0.2483 - lr: 0.0100
Epoch 6/10
1337/1337 [==============================] - 2s 1ms/step - loss: 2.7975 - accuracy: 0.2481 - lr: 0.0100
Epoch 7/10
1337/1337 [==============================] - 2s 1ms/step - loss: 2.7825 - accuracy: 0.2484 - lr: 1.0000e-03
Epoch 8/10
1337/1337 [==============================] - 2s 2ms/step - loss: 2.7775 - accuracy: 0.2539 - lr: 1.0000e-03
Epoch 9/10
1337/1337 [==============================] - 2s 2ms/step - lo

### Evaluation

In [82]:
object_result = object_cls.evaluate(object_x_test, object_y_test)
print("test_loss:", object_result[0], "test_accuracy:", object_result[1])

84/84 [==============================] - 0s 1ms/step - loss: 2.7710 - accuracy: 0.2551
test_loss: 2.7710049152374268 test_accuracy: 0.25514402985572815


## #TODO 5: Build and evaluate a multi-task model that does both "action" and "object" classifications in one-go 

This can be a bit tricky, if you are not familiar with the Keras functional API. PLEASE READ these webpages(https://www.tensorflow.org/guide/keras/functional, https://keras.io/getting-started/functional-api-guide/) before you start this task.   

Your model will have 2 separate output layers one for action classification task and another for object classification task. 

This is a rough sketch of what your model might look like:
image --> https://drive.google.com/file/d/1r7M6tFyQDu6pJIxLd_fn2kBMjo_CWmUK/view?usp=share_link

In [83]:
#TODO 5: Build and evaluate a multi-task model that does both "action" and "object" classifications in one-go

### Create Model

In [84]:
def combine_Model():
    action_inputs = Input(shape=(action_x_train.shape[1],), name="action")
    object_inputs = Input(shape=(object_x_train.shape[1],), name="object")

    #Action model
    action_x = Dense(units=32, activation='relu')(action_inputs)
    action_x = Dropout(rate=0.2)(action_x)
    action_x = Dense(units=16, activation='relu')(action_x)
    
    action_x = Dense(units=action_y_train.shape[1], activation='relu')(action_x)
    action_pred = Activation('softmax', name="action_pred")(action_x)

    #Object model
    object_x = Dense(units=32, activation='relu')(object_inputs)
    object_x = Dropout(rate=0.2)(object_x)
    object_x = Dense(units=16, activation='relu')(object_x)
    
    object_x = Dense(units=object_y_train.shape[1], activation='relu')(object_x)
    object_pred = Activation('softmax', name="object_pred")(object_x)

    return Model(inputs=[action_inputs, object_inputs], 
                 outputs=[action_pred, object_pred])

In [85]:
combine_cls = combine_Model()
combine_cls.summary()
# plot_model(combine_model)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 action (InputLayer)            [(None, 15)]         0           []                               
                                                                                                  
 object (InputLayer)            [(None, 15)]         0           []                               
                                                                                                  
 dense_32 (Dense)               (None, 32)           512         ['action[0][0]']                 
                                                                                                  
 dense_35 (Dense)               (None, 32)           512         ['object[0][0]']                 
                                                                                            

### Train Model

In [86]:
combine_opt = Adam(learning_rate=0.01)
combine_cls.compile(optimizer=object_opt, 
                   loss={"action_pred": "categorical_crossentropy",
                         "object_pred": "categorical_crossentropy"},
                   metrics=['accuracy'])

In [87]:
combine_scheduler = ReduceLROnPlateau(monitor='action_pred_loss', factor=0.1, patience=1, min_lr=0.0001)
combine_cls.fit({"action": action_x_train, "object": object_x_train},
                {"action_pred": action_y_train, "object_pred": object_y_train},
                epochs=10,
                batch_size=8,
                callbacks=combine_scheduler)

Epoch 1/10
1337/1337 [==============================] - 4s 2ms/step - loss: 4.2597 - action_pred_loss: 1.3670 - object_pred_loss: 2.8927 - action_pred_accuracy: 0.6421 - object_pred_accuracy: 0.1679 - lr: 1.0000e-04
Epoch 2/10
1337/1337 [==============================] - 3s 2ms/step - loss: 3.6927 - action_pred_loss: 1.0959 - object_pred_loss: 2.5968 - action_pred_accuracy: 0.7030 - object_pred_accuracy: 0.2622 - lr: 1.0000e-04
Epoch 3/10
1337/1337 [==============================] - 3s 2ms/step - loss: 3.5401 - action_pred_loss: 1.0355 - object_pred_loss: 2.5046 - action_pred_accuracy: 0.7075 - object_pred_accuracy: 0.2991 - lr: 1.0000e-04
Epoch 4/10
1337/1337 [==============================] - 3s 2ms/step - loss: 3.4609 - action_pred_loss: 1.0123 - object_pred_loss: 2.4486 - action_pred_accuracy: 0.7081 - object_pred_accuracy: 0.3138 - lr: 1.0000e-04
Epoch 5/10
1337/1337 [==============================] - 3s 2ms/step - loss: 3.4134 - action_pred_loss: 1.0006 - object_pred_loss: 2.4128

### Evaluation

In [88]:
combine_result = combine_cls.evaluate({"action": action_x_test, "object": object_x_test},
                                    {"action_pred": action_y_test, "object_pred": object_y_test})

print("action loss:", combine_result[1], "action accuracy", combine_result[3])
print("object loss:", combine_result[2], "object accuracy", combine_result[4])

84/84 [==============================] - 1s 2ms/step - loss: 3.2409 - action_pred_loss: 0.9446 - object_pred_loss: 2.2963 - action_pred_accuracy: 0.7198 - object_pred_accuracy: 0.3427
action loss: 0.9445986151695251 action accuracy 0.719790518283844
object loss: 2.2962758541107178 object accuracy 0.34268611669540405


## #Using USE (Multilingual Universal Sentence Encoder) 
** just wanna try **

## #Summary all of this notebook